In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import time
import cPickle
import os
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

In [2]:
def getFrameData(root, iterKey):
    allFrameData = []
    for frameData in root.iter(iterKey):
        if not frameData.attrib:
            allFrameData.append(frameData.text)
        else:
            allFrameData.append(frameData.attrib)
    return allFrameData

def readRuleData(file_name):
    tree = ET.parse(file_name)
    root = tree.getroot()
    parent = {}
    for child in root:
        tempList = list(child)
    #     print child.tag
        if not tempList:
            if not child.attrib:
                parent[child.tag] = child.text
            else:
                parent[child.tag] = child.attrib
        else:
            parent[child.tag] = {}
            for grand_child in child:
    #             print '\t', grand_child.tag
                tempList = list(grand_child)
                if not tempList:
                    if not grand_child.attrib:
                        parent[child.tag][grand_child.tag] = grand_child.text
                    else:
                        parent[child.tag][grand_child.tag] = grand_child.attrib
                else:
                    parent[child.tag][grand_child.tag] = {}
                    for grand_grand_child in grand_child:
    #                     print '\t\t', grand_grand_child.tag
                        tempList = list(grand_grand_child)
                        if not tempList:
                            if not grand_grand_child.attrib:
                                parent[child.tag][grand_child.tag][grand_grand_child.tag] = grand_grand_child.text
                            else:
                                parent[child.tag][grand_child.tag][grand_grand_child.tag] = grand_grand_child.attrib
                        else:
                            parent[child.tag][grand_child.tag][grand_grand_child.tag] = {}
                            for grand_grand_grand_child in grand_grand_child:
    #                             print '\t\t\t', grand_grand_grand_child.tag
                                tempList = list(grand_grand_grand_child)
                                if not tempList:
                                    if not grand_grand_grand_child.attrib:
                                        parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.text
                                    else:
                                        parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.attrib
                                else:
                                    parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag] = {}
                                    for grand_grand_grand_grand_child in grand_grand_grand_child:
            #                             print '\t\t\t', grand_grand_grand_grand_child.tag
                                        tempList = list(grand_grand_grand_grand_child)
                                        if not tempList:
                                            if not grand_grand_grand_grand_child.attrib:
                                                parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.text
                                            else:
                                                parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.attrib
                                        else:
                                            print "There are more grand childrens than you think"
    parent['ruleData'] = []
    parent['ruleData'] = pd.DataFrame(getFrameData(root, 'ruleData'))
    
    return parent

def readKinData(file_name, inum):
    print 'Parsing File Number ', inum
    tree = ET.parse(file_name)
    root = tree.getroot()
    
    kinData = []
    kinetics = []
    stimulus = []

    larvaFrameData = pd.DataFrame(getFrameData(root, 'larvaFrameData'))
    skeleton = pd.DataFrame(getFrameData(root, 'skeleton'))
    kinetics = pd.concat([larvaFrameData.reset_index(drop=True), skeleton.reset_index(drop=True)], axis=1)

    midpoint = pd.DataFrame(getFrameData(root, 'midpoint'), dtype=np.float16)
    midpoint.rename(columns={'x': 'mid_x', 'y': 'mid_y'}, inplace=True)
    midpoint.reset_index(inplace=True, drop=True)
    centroid = pd.DataFrame(getFrameData(root, 'centroid'), dtype=np.float16)
    centroid.rename(columns={'x': 'cent_x', 'y': 'cent_y'}, inplace=True)
    centroid.reset_index(inplace=True, drop=True)
    head = pd.DataFrame(getFrameData(root, 'head'), dtype=np.float16)
    head.rename(columns={'x': 'head_x', 'y': 'head_y'}, inplace=True)
    head.reset_index(inplace=True, drop=True)
    tail = pd.DataFrame(getFrameData(root, 'tail'), dtype=np.float16)
    tail.rename(columns={'x': 'tail_x', 'y': 'tail_y'}, inplace=True)
    tail.reset_index(inplace=True, drop=True)    
    featureLocation = pd.DataFrame(getFrameData(root, 'featureLocation'), dtype=np.float16)
    s = featureLocation.copy()
    all_feature_df = pd.DataFrame([])
    all_feat = s.name.unique()
    for feat in all_feat:
        d = s.loc[s['name'] == feat, ['numberMaximumVotes', 'numberVotes', 'x', 'y']]
        d.rename(columns={'numberMaximumVotes': feat + '_maxVotes', 
                          'numberVotes': feat + '_votes',
                          'x': feat + '_x',
                          'y': feat + '_y'}, inplace=True)
        all_feature_df = pd.concat([all_feature_df, d.reset_index(drop=True)], axis=1)
    
    stimulusIntensity = pd.DataFrame(getFrameData(root, 'intensityPercentage'))
    temp = pd.DataFrame(getFrameData(root, 'ledArrayStimulus'))
    
    if not temp.empty:
        stimulus = pd.concat([temp.reset_index(drop=True), stimulusIntensity.reset_index(drop=True)], axis=1)
        stimulus.rename(columns={0: 'intensity'}, inplace=True)
    else:
        stimulus = stimulusIntensity
    
    kinData = pd.concat([kinetics.reset_index(drop=True), midpoint, centroid, head, tail, all_feature_df, stimulus.reset_index(drop=True)], axis=1)
    kinData['num'] = inum
    kinData['timeIndex'] = kinData.index.values

    return kinData

In [3]:
# data_dir = '20180316_OK6_CsChrimson_Tail_200micron'
data_dir = '20180316_OK6_CsChrimson_Tail_200micron'

all_xml_files = []
root_path = '/Volumes/GoogleDrive/My Drive/CRG_Dropbox/AljoComputer/Matlab/high_res_tracker/Data/'
data_dir = os.path.join(root_path,data_dir,'XML')
for file in os.listdir(data_dir):
    if file.endswith(".xml"):
        print file
        all_xml_files.append(os.path.join(data_dir, file))

print "Total XML Files Found - ", len(all_xml_files)

venkman-log-20180316-062750100-sid-1.xml
venkman-log-20180316-063825616-sid-2.xml
Total XML Files Found -  2


In [4]:
## Parse Files in Parallel
tStart = time.time()
kin_data = []
kin_data = Parallel(n_jobs=num_cores)(delayed(readKinData)(xml_file, idx) for idx, xml_file in enumerate(all_xml_files))
print 'Time taken to parse %d files in parallel is %0.3f sec'%(len(all_xml_files), time.time()-tStart)

Parsing File Number  0
Parsing File Number  1
Time taken to parse 2 files in parallel is 35.543 sec


In [6]:
## Parse Files in Parallel
tStart = time.time()
rule_data = []
rule_data = readRuleData(all_xml_files[0])
print 'Time taken to parse rule data is %0.3f sec' %(time.time()-tStart)

Time taken to parse rule data is 22.760 sec


In [7]:
kin_data_all=[]
kin_data_all = pd.concat(kin_data)
kin_data_all.reset_index(inplace=True)

## Save Parsed Files
kin_data_all.to_csv(open(os.path.join(data_dir, 'kinData_xml.dat'), 'wb'), sep='\t', index=False)
cPickle.dump(rule_data, open(os.path.join(data_dir, 'ruleData_xml.pkl'), 'wb'), protocol=2)

In [ ]:
## Parse Files Sequentially
# c=0
# for xml_file in all_xml_files:
#     tStart = time.time()
#     temp = readXMLFile(xml_file)
#     kin_data.append(readXMLFile(xml_file))
#     c+=1
#     print 'Time taken to parse file %d is %0.3f'%(c, time.time()-tStart)